In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras

#from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import pylab
#plt.rcParams['figure.figsize'] = 10, 10
#%matplotlib inline

Using TensorFlow backend.


In [2]:
train = pd.read_json("_RawData/train.json/data/processed/train.json")
target_train=train['is_iceberg']
test = pd.read_json("_RawData/test.json/data/processed/test.json")


In [3]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [4]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#X_band_3=(X_band_1+X_band_2)/2
X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
X_band_4=np.maximum(X_band_1,X_band_2)
X_band_5=np.minimum(X_band_1,X_band_2)
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([
                          
                          X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_band_test_3=np.fabs(np.subtract(X_band_test_1,X_band_test_2))
X_band_test_4=np.maximum(X_band_test_1,X_band_test_2)
X_band_test_5=np.minimum(X_band_test_1,X_band_test_2)
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([
                          X_band_test_3[:, :, :, np.newaxis], X_band_test_4[:, :, :, np.newaxis],X_band_test_5[:, :, :, np.newaxis]],axis=-1)


In [5]:
#Import Keras.
#from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=32
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    return genX1

# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


# def getVggAngleModel():
#     input_2 = Input(shape=[1], name="angle")
#     angle_layer = Dense(1, )(input_2)
#     base_model = VGG16(weights='imagenet', include_top=False, 
#                  input_shape=X_train.shape[1:], classes=1)
#     x = base_model.get_layer('block5_pool').output
#     x = GlobalMaxPooling2D()(x)
#     base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

#     x2 = base_model2.output
#     x2 = GlobalAveragePooling2D()(x2)

#     merge_one = concatenate([x, x2, angle_layer])

#     merge_one = Dropout(0.6)(merge_one)
#     predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
#     model = Model(input=[base_model.input, input_2], output=predictions)
    
#     sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
#     model.compile(loss='binary_crossentropy',
#                   optimizer=sgd,
#                   metrics=['accuracy'])
#     return model

sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

def getVggAngleModel():
    img_input = Input(shape=(75, 75, 3))
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(1, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('sigmoid', name='loss')(x)
    
    model = Model(img_input, x, name='squeezenet')
#     model = SqueezeNet(include_top=True, weights = None, classes = 1)
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
#     print(model.summary())
    return model

#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        
        print(X_holdout.shape)
        print(X_angle_hold.shape)

        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=(X_holdout, Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate(X_train_cv, y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate(X_holdout, Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict(X_holdout)
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [6]:
# preds=myAngleCV(X_train, X_angle, X_test)
# #Submission for each day.
# submission = pd.DataFrame()
# submission['id']=test['id']
# submission['is_iceberg']=preds
# submission.to_csv('subvgg16mobile_squeeze.csv', index=False)

In [7]:
from models import DaveBaseModel
import keras_squeezenet
import importlib
importlib.reload(keras_squeezenet)
from keras_squeezenet import SqueezeNet

class Squeeze(DaveBaseModel):
    def get_model(self):
#         img_input = Input(shape=(75, 75, 3))

#         x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
#         x = Activation('relu', name='relu_conv1')(x)
#         x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

#         x = fire_module(x, fire_id=2, squeeze=16, expand=64)
#         x = fire_module(x, fire_id=3, squeeze=16, expand=64)
#         x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

#         x = fire_module(x, fire_id=4, squeeze=32, expand=128)
#         x = fire_module(x, fire_id=5, squeeze=32, expand=128)
#         x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

#         x = fire_module(x, fire_id=6, squeeze=48, expand=192)
#         x = fire_module(x, fire_id=7, squeeze=48, expand=192)
#         x = fire_module(x, fire_id=8, squeeze=64, expand=256)
#         x = fire_module(x, fire_id=9, squeeze=64, expand=256)

#         x = Dropout(0.5, name='drop9')(x)

#         x = Convolution2D(1, (1, 1), padding='valid', name='conv10')(x)
#         x = Activation('relu', name='relu_conv10')(x)
#         x = GlobalAveragePooling2D()(x)
#         x = Activation('sigmoid', name='loss')(x)

#         model = Model(img_input, x, name='squeezenet')
#     #     model = SqueezeNet(include_top=True, weights = None, classes = 1)
#         sgd = Adam(lr=1e-5) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

#         model.compile(loss='binary_crossentropy',
#                       optimizer=sgd,
#                       metrics=['accuracy'])

    #     print(model.summary())
    
        model = SqueezeNet(weights = None, classes = 1, input_shape=(75, 75, 3))
        sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

        model.compile(loss='binary_crossentropy',
                      optimizer=sgd,
                      metrics=['accuracy'])
        return model

    def get_name(self):
        return "squeeze"

In [8]:
model = Squeeze(test['id'])

In [9]:
model.train(X_train, target_train)



Model: squeeze
Batch Size: 32
Epochs: 50
Epoch 1/50
51/50 [==============================] - 7s 138ms/step - loss: 8.5171 - acc: 0.4658
Epoch 2/50
51/50 [==============================] - 2s 40ms/step - loss: 8.5171 - acc: 0.4658
Epoch 3/50
51/50 [==============================] - 2s 44ms/step - loss: 8.4500 - acc: 0.4700
Epoch 4/50
51/50 [==============================] - 2s 39ms/step - loss: 8.4500 - acc: 0.4700: 0s - loss: 8.2895 - acc
Epoch 5/50
51/50 [==============================] - 2s 41ms/step - loss: 8.5171 - acc: 0.4658
Epoch 6/50
51/50 [==============================] - 2s 43ms/step - loss: 8.4500 - acc: 0.4700
Epoch 7/50
51/50 [==============================] - 2s 41ms/step - loss: 8.5171 - acc: 0.4658
Epoch 8/50
51/50 [==============================] - 2s 41ms/step - loss: 8.4500 - acc: 0.4700
Epoch 9/50


KeyboardInterrupt: 